In [1]:
import numpy
import optuna
from pycbc.filter import matched_filter
from pycbc.waveform import get_td_waveform
from pycbc.catalog import Merger
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation

/home/susanna.green/.conda/envs/PyCBCandOptuna/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
merger = Merger("GW170814")

# Get the data from the Hanford detector
strain = merger.strain('V1')

# Remove the low frequency content and downsample the data to 2048Hz
strain = resample_to_delta_t(highpass(strain, 15.0), 1.0/2048)

# Remove 2 seconds of data from both the beginning and end
conditioned = strain.crop(2, 2)

# Estimate the power spectral density
# We use 4 second samles of our time series in Welch method.
psd = conditioned.psd(4)

# Now that we have the psd we need to interpolate it to match our data
# and then limit the filter length of 1 / PSD. After this, we can
# directly use this PSD to filter the data in a controlled manner

psd = interpolate(psd, conditioned.delta_f)

# 1/PSD will now act as a filter with an effective length of 4 seconds
# Since the data has been highpassed above 15 Hz, and will have low values
# below this we need to informat the function to not include frequencies
# below this frequency. 
psd = inverse_spectrum_truncation(psd, int(4 * conditioned.sample_rate),low_frequency_cutoff=15)

In [3]:
#Optuna gnerating the template. 
def objective(trial):
    m1 = trial.suggest_float('m1', 5, 100)
    m2 = trial.suggest_float('m2', 5, 100)
    hp, hc = get_td_waveform(approximant="SEOBNRv4_opt",mass1=m1, mass2=m2, delta_t=conditioned.delta_t,f_lower=20)
    hp.resize(len(conditioned))
    template = hp.cyclic_time_shift(hp.start_time)
    snr = matched_filter(template, conditioned, psd=psd, low_frequency_cutoff=20)
    snr = snr.crop(4 + 4, 4)
    peak = abs(snr).numpy().argmax()
    snrp = abs(snr[peak])
    return float(snrp)


In [5]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_params)

[I 2023-03-01 12:49:53,388] A new study created in memory with name: no-name-3db5bd74-786f-469b-b1b9-ab74a93d461e
[I 2023-03-01 12:49:53,505] Trial 0 finished with value: 4.252930612320342 and parameters: {'m1': 76.9405263239678, 'm2': 87.00886946267502}. Best is trial 0 with value: 4.252930612320342.
[I 2023-03-01 12:49:53,638] Trial 1 finished with value: 4.33433395857878 and parameters: {'m1': 49.41571669889552, 'm2': 90.90411569462977}. Best is trial 1 with value: 4.33433395857878.
[I 2023-03-01 12:49:53,764] Trial 2 finished with value: 4.558906449296799 and parameters: {'m1': 25.79058935871872, 'm2': 81.99167900760865}. Best is trial 2 with value: 4.558906449296799.
[I 2023-03-01 12:49:53,901] Trial 3 finished with value: 4.407767713966755 and parameters: {'m1': 85.12624274117897, 'm2': 66.18115705158793}. Best is trial 2 with value: 4.558906449296799.
[I 2023-03-01 12:49:54,030] Trial 4 finished with value: 5.13492323481249 and parameters: {'m1': 25.179249272288324, 'm2': 15.028

RuntimeError: Internal function call failed: Input domain error